In [150]:
##
# Aplica condições de contorno essenciais 
# homogêneas
#
function Aplica_CC_homo!(apoios,K,F)

    # Aplica as condições de contorno essenciais homogêneas
    for i=1:size(apoios,1) 

        # No e gl local do apoio
        no  = Int(apoios[i,1])
        gll = Int(apoios[i,2])

        # Testa se o usuário tem ideia do que
        # ele está fazendo
        if apoios[i,3]!=0 
            error("Sua anta")
        end

        # Gl global
        gl = 2*(no-1) + gll

        # Zera linha e coluna da rigidez
        K[gl,:] .= 0.0
        K[:,gl] .= 0.0

        # Zera o vetor de forças nesse gl
        F[gl] = 0.0

        # Coloca 1.0 na diagonal
        K[gl,gl] = 1.0

    end #gls

end


#
# Aplica cc essenciais por Multiplicadores de Lagrange
#
# Monta o sistema aumentado
# [K S' {U}= {F}
#  S 0] {L}  {Ub}
#
#
function Aplica_CC_lagrange(nnos,apoios,K,F)

    # Número de gls do problema original
    n = 2*nnos

    # Número de cc essenciais
    m = size(apoios,1)

    # Define o sistema aumentado de equações
    KA = zeros(n+m,n+m)
    FA = zeros(n+m)

    # Define a matriz S e o vetor Ub
    S  = zeros(m,n)
    Ub = zeros(m)

    # Aplica as condições de contorno essenciais
    for i=1:m 

        # No e gl local do apoio
        no  = Int(apoios[i,1])
        gll = Int(apoios[i,2])
        valor = apoios[i,3]

        # Gl global
        gl = 2*(no-1) + gll

        # Posiciona na linha da matriz S
        S[i,gl] = 1.0

        # Posiciona o valor em Ub
        Ub[i] = valor

    end

    # Posiciona os blocos na matriz e no vetor aumentados
    KA[1:n,1:n]     .= K
    KA[1:n,n+1:end] .= S'
    KA[n+1:end,1:n] .= S

    FA[1:n]     .= F
    FA[n+1:end] .= Ub

    return KA, FA
end

Aplica_CC_lagrange (generic function with 1 method)

In [151]:
#
# conectividades é uma matriz com ne linhas (número de elementos)
# e 2 colunas (nó inicial e nó final)
#
function Global(ne,nnos,conectividades,coord, VE,Vnuxy, esp)

    # Precisamos definir a matriz global
    K = zeros(2*nnos,2*nnos)

    # Aloca vetores para as coordenadas de cada elemento
    X = zeros(4)
    Y = zeros(4)
    nos = zeros(4)
    gls = zeros(Int,8)

    # Loop nos elementos da malha
    for ele=1:ne

        # Recupera as informações do elemento
        Ee = VE[ele]
        ve = Vnuxy[ele]

        # Monta a matriz constitutiva do elemento
        C = Calcula_C(Ee,ve,"EPD")

        # Recupera as coordenadas do elemento
        for i=1:4
            # Descobre o nó
            no = conectividades[ele,i]
            
            # Guarda os nós do elemento
            nos[i] = no

            # Descobre as coordenadas
            X[i],Y[i] = coord[no,:]
        end

        # Monta a matriz de rigidez do elemento
        # no sistema local
        Kg = K_bi4n(C,X,Y,esp)

        # Agora precisamos posicionar Kg na matriz global do problema

        # Vetor com os gls GLOBAIS do elemento
        contador = 1
        # Varre os nós
        for i=1:4
            # Varre os gls do nó
            for j=1:2
                gls[contador] = Int(2*(nos[i]-1)+j)
                contador += 1
            end
        end

        # Soma Kg nas posições gls em K
        K[gls,gls] .= K[gls,gls] .+ Kg

    end #ele

    # Retorna a matriz de rigidez do problema
    return K

end


#
# Monta o vetor de força global
#
function Forca_global(nnos,forcas)

    # Montar o vetor de forças global
    F = zeros(2*nnos)

    # Para cada informação em forças 
    # posiciona a força no vetor de forças
    # globais do problema
    for i=1:size(forcas,1)
       
        # Recupera nó e gl local 
        no  = Int(forcas[i,1])
        gll = Int(forcas[i,2])

        # Recupera valor
        valor = forcas[i,3]

        # Adiciona ao valor da força
        F[2*(no-1)+gll] += valor

    end #i

    return F
end

Forca_global (generic function with 1 method)

In [152]:

# Monta a matriz constitutiva dado E,v e a hipótese
function Calcula_C(E,v,hipotese="EPD")

    # Testa se hipotese é válida
    hipotese in ["EPT","EPD"] || throw("Hipotese errada")

    G = E/(2*(1+v))
       
    if hipotese=="EPT"
        a = E/(1-v^2)
        C = [a v*a 0.0 ;
            v*a a  0.0 ;
              0  0  G]

    elseif hipotese=="EPD"
        d = (v+1)*(2*v-1)  
        a = E*(v-1)/d
        b = -(E*v)/d
        C = [a b 0 ;
             b a 0 ;
             0 0 G]
    else
 
        C = [a b b 0 0 0 ;
             b a b 0 0 0 ;
             b b a 0 0 0 ;
             0 0 0 G 0 0 ;
             0 0 0 0 G 0 ;
             0 0 0 0 0 G]

    end
    
    return C

end

Calcula_C (generic function with 2 methods)

In [153]:
   # N1 = 0.25(1-r)(1-s)
   # N2 = 0.25(1+r)(1-s)
   # N3 = 0.25(1+r)(1+s)
   # N4 = 0.25(1-r)(1+s)


#
# Calcula a matriz B e o determinante da matriz
# Jacobiana no ponto (r,s) 
#
function B_bi4n(r,s,X,Y)

   # Vetores com as derivadas das funções de interpolação
   # em relação a r e a s
   dNr = [ -0.25*(1-s) ; 0.25*(1-s) ; 0.25*(1+s)  ; -0.25*(1+s)   ]
   dNs = [ -0.25*(1-r) ;-0.25*(1+r) ; 0.25*(1+r)  ;  0.25*(1-r)   ]  
   nnos = length(dNr) 

   # Aloca e calcula a matriz J
   J = zeros(2,2)

   # Loop do somatório para cada posição de J, i depende da quantidade de nós do elemento
   for i=1:nnos
       J[1,1] += dNr[i]*X[i]
       J[1,2] += dNr[i]*Y[i]
       J[2,1] += dNs[i]*X[i]
       J[2,2] += dNs[i]*Y[i]
   end #i

   # Calcula o determinante
   dJ = det(J)

   # Calcula a inversa da J
   invJ = inv(J)

   # Aloca a matriz B
   B = zeros(3,2*nnos)


   # Loop pelos blocos da B, com as correções de derivadas
   c = 1
   for i=1:nnos
     
       # Correção das derivadas para este bloco
       dNxy = invJ*[dNr[i];dNs[i]]

       # Posiciona na B
       B[1,c]   = dNxy[1]
       B[2,c+1] = dNxy[2]
       B[3,c]   = dNxy[2]
       B[3,c+1] = dNxy[1]

       # Atualiza a apontador de bloco
       c += 2

   end #i
   return dJ, B
end



#
# Monta a matriz de rigidez de um elemento
# bilinear isoparamétrico de 4 nós
#
function K_bi4n(C::AbstractMatrix,X::Vector,Y::Vector,esp::Float64)

    # Aloca a matriz do elemento
    K = zeros(8,8)

    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]

    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
           s  = pg[j]  
           ws = W[j]

           # calcula o dJ e o B
           dJ, B = B_bi4n(r,s,X,Y)

           # Acumula o produto nesses ptos
           K = K + B'*C*B*dJ*esp

        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return K

end


K_bi4n (generic function with 1 method)

In [154]:
using LinearAlgebra

function main2(func::Function, v0)

    nnos,ne,dim,coord,conectividades,VE,Vnuxy,apoios,forcas = func(v0)
    esp = 1.0
    
    # Monta a matriz global do problema
    K = Global(ne,nnos,conectividades,coord, VE, Vnuxy, esp)
    sparse(K)

    # Monta o vetor de forças globais concentradas
    F = Forca_global(nnos,forcas)

    # Modifica o sistema pela aplicação das condições de contorno
    # homogêneas
    Aplica_CC_homo!(apoios,K,F)
    #KA, FA = Aplica_CC_lagrange(nnos,apoios,K,F)
    
    # Soluciona o sistema de equações, obtendo os deslocamentos
    U = K\F
    #UA = KA\FA

    # Só os deslocamentos que nos interessam
    #U = UA[1:2*nnos]

    # Retorna U
    return U , K

end


main2 (generic function with 1 method)

In [155]:
using NBInclude
@nbinclude("casos.ipynb")

caso_g2 (generic function with 1 method)

In [156]:
U , K = main2(caso_g2,0.49)

([0.0, 0.0, 5.258315735487588, -1.260161619697545, 18.444677286596605, -2.396197213947522, 36.99283231438755, -2.831869746865793, 0.0, 0.0  …  37.0037374969663, 2.8988478102617528, 0.0, 0.0, 5.1416698013749995, 0.0018573940077363797, 18.395405859065498, 0.00974223111550859, 36.97356721351419, 0.03320797608917185], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 380480.9843400444 -3.637978807091713e-12; 0.0 0.0 … -3.637978807091713e-12 16014.168530947045])

In [157]:
imprimir(U)

Nó	Deslocamento em x	Deslocamento em y
1	   0.00000	             0.00000
2	   5.25832	            -1.26016
3	  18.44468	            -2.39620
4	  36.99283	            -2.83187
5	   0.00000	             0.00000
6	   5.25950	             1.26401
7	  18.44846	             2.41605
8	  37.00374	             2.89885
9	   0.00000	             0.00000
10	   5.14167	             0.00186
11	  18.39541	             0.00974
12	  36.97357	             0.03321


In [158]:
using Printf

function imprimir(deslocamentos)
    n_nos = length(deslocamentos) ÷ 2
    # Cabeçalho da tabela
    println("Nó\tDeslocamento em x\tDeslocamento em y")
    for i in 1:n_nos
        desloc_x = deslocamentos[2*i-1]
        desloc_y = deslocamentos[2*i]
        @printf("%d\t%10.5f\t%20.5f\n", i, desloc_x, desloc_y)
    end
end



imprimir (generic function with 1 method)